In [3]:
import os
from openai import OpenAI

OPENAI_API_KEY = os.environ['OPENAI_API_KEY']

client = OpenAI(api_key=OPENAI_API_KEY)


In [ ]:
compile = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Hi"},],
    temperature=0.0
)
print(compile.choices[0].message.content)

Hello! How can I assist you today?


### 데이터 전처리 코드

리뷰 JSON 파일 로드

In [1]:
import json

with open('./res/reviews.json', 'r') as f:
    review_list = json.load(f)
review_list[:3]

[{'review': '너무좋습니다 감사합니다', 'stars': 5, 'date': '2분 전'},
 {'review': '가족들과 좋은시간 보내고 왔습니다.\n부대시설 좋고 수영장도 좋습니다.\n앞으로도 종종 이용할 것 같습니다!\n\n날씨 좋을 때 한번 더 방문할께요!',
  'stars': 5,
  'date': '3시간 전'},
 {'review': '청결하고 좋았습니다.', 'stars': 5, 'date': '15시간 전'}]

좋은 평점 및 나쁜 평점 정의
- 좋은 평점 = 별 5개
- 나쁜 평점 = 별 4개 이하

In [2]:
good_cnt, bad_cnt = 0, 0
for r in review_list:
    if r['stars'] == 5:
        good_cnt += 1
    else:
        bad_cnt += 1

good_cnt, bad_cnt

(189, 31)

좋은 리뷰 및 나쁜 리뷰로 구분 후 저장

In [3]:
good_reviews, bad_reviews = [], []
for r in review_list:
    if r['stars'] == 5:
        good_reviews.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')
    else:
        bad_reviews.append('[REVIEW_START]' + r['review'] + '[REVIEW_END]')

bad_reviews[:3]

['[REVIEW_START]시설적이 부분에서는 딱히 흠잡을게 없었고 가격대가 좀 있지만 퀄리티가 괜찮은 편이라 so so인데 유독 스파 직원들이 불친절함 표정도 그렇고 말투나 톤이 뭔가 하기 싫다는 듯한 느낌을 받음[REVIEW_END]',
 '[REVIEW_START]직원친절도 최상.  숙소 정비나 시설이 깨끗하고 뒷산뷰도 조용하고 좋았습니다.[REVIEW_END]',
 '[REVIEW_START]깨끗하고 산책하기도 좋아요. 애들이랑 물놀이하기 좋구요. 재미있게 놀다왔습니다.[REVIEW_END]']

In [4]:
good_reviews_text = '\n'.join(good_reviews)
bad_reviews_text = '\n'.join(bad_reviews)

전처리 함수 작성

In [ ]:
def preprocess_reviews(path = './res/reviews.json'):
    